In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

import numpy as np
import matplotlib.pyplot as plt
import os
import tqdm

competition_path = "/content/gdrive/My Drive/Seismic Facies Identification/"
binary_pred_path = competition_path + "binary_predictions/"
os.listdir(binary_pred_path)

In [2]:
data_path = competition_path + "data/"
test2_img = np.load(data_path+'data_test_2.npz', allow_pickle=True, mmap_mode='r')['data']

In [3]:
# moving average
def moving_average(pred, axis=1, window=71): # window: odd number
  new_pred = np.zeros_like(pred, dtype=float)
  n_images = pred.shape[axis]
  for i in range(n_images):
    side_images = min(int((window-1)/2), i, n_images-i-1)
    if axis==1:
      new_pred[:,i,:] = np.mean(pred[:,i-side_images:i+side_images+1], axis=axis)
    else: # axis==2
      new_pred[:,:,i] = np.mean(pred[:,:,i-side_images:i+side_images+1], axis=axis)
  print("Moving average function finished.")
  return np.round(new_pred).astype(np.int8)

In [6]:
class0123_file = "class0123_02_best.npz"
class4_file = "class4_20_CE0.3_GDice0.7_weight10.npz"
class5_file = "class5_04_CE0.3_GDice0.7_w5_train19.npz"
class5_file_bis = "class5_04_CE0.3_GDice0.7_w5_train14.npz"

pred_class0123 = np.load(binary_pred_path+class0123_file, mmap_mode='r')['prediction']
pred_class0123 -= 1
pred_class4 = np.load(binary_pred_path+class4_file, mmap_mode='r')['prediction']
pred_class4 -= 1
pred_class5 = np.load(binary_pred_path+class5_file, mmap_mode='r')['prediction']
pred_class5 -= 1
pred_class5_bis = np.load(binary_pred_path+class5_file_bis, mmap_mode='r')['prediction']
pred_class5_bis -= 1

pred_class5 = np.logical_or(pred_class5==5, pred_class5_bis==5)

In [ ]:
# moving average
pred_class0 = moving_average(pred_class0123==0)
pred_class2 = moving_average(pred_class0123==2)
pred_class3 = moving_average(pred_class0123==3)
pred_class4 = moving_average(pred_class4==4)
pred_class5 = moving_average(pred_class5==1)

# join classes together
pred = np.ones_like(pred_class0123) 
pred[np.logical_and(pred==1, pred_class0==1)] = 0
pred[np.logical_and(pred==1, pred_class2==1)] = 2
pred[np.logical_and(pred==1, pred_class3==1)] = 3
pred[np.logical_and(pred==1, pred_class4==1)] = 4
pred[np.logical_and(pred==1, pred_class5==1)] = 5

In [8]:
from google.colab import files

submission_file = competition_path+'submissions/prediction54.npz'
np.savez_compressed(
    submission_file,
    prediction=pred + 1
)

files.download(submission_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# plot image & predictions
def plot_image_pred(image, pred=None, labels=None):

  # plot image & labels
  fig, ax = plt.subplots(1,3, sharey=True)
  fig.set_size_inches(20, 8)
  fig.suptitle("2D slice of the 3D seismic data volume", fontsize=20)
  ax[0].imshow(image, cmap='terrain')
  ax[0].set_xlabel('X or Y Axis', fontsize=14)
  ax[0].set_ylabel('Z Axis: Top - Bottom', fontsize=14)
  ax[0].set_title("image", fontsize=20)
  if labels is not None:
    print(np.unique(labels))
    ax[1].imshow(labels)
    ax[1].set_xlabel('X or Y Axis', fontsize=14)
    ax[1].set_title("labels", fontsize=20)
    ax[2].imshow(image, cmap='terrain')
    ax[2].imshow(labels, alpha=0.4, cmap='twilight')
    ax[2].set_xlabel('X or Y Axis', fontsize=14)
    ax[2].set_title("image + labels", fontsize=20)

  # plot pred & labels
  if pred is not None:
    fig, ax = plt.subplots(1,3, sharey=True)
    fig.set_size_inches(20, 8)
    fig.suptitle("Predictions & labels", fontsize=20)
    ax[0].imshow(pred)
    ax[0].set_xlabel('X or Y Axis', fontsize=14)
    ax[0].set_title("predictions", fontsize=20)
    if labels is not None:
      ax[1].imshow(labels)
      ax[1].set_xlabel('X or Y Axis', fontsize=14)
      ax[1].set_title("labels", fontsize=20)
      ax[2].imshow(pred.equal(labels))
      ax[2].set_xlabel('X or Y Axis', fontsize=14)
      ax[2].set_title("wrong predictions", fontsize=20)

plot_image_pred(test2_img[:,:,0], pred[:,:,0])
plot_image_pred(test2_img[:,75,:], pred[:,75,:])